In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 20      # number of variables
num_ineq = 20     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("scip")
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/nc_exact_10-10.csv")

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

  1%|▊                                                                                 | 1/100 [00:03<05:22,  3.26s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

  2%|█▋                                                                                | 2/100 [00:04<02:57,  1.81s/it]

...

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [31:15<00:18, 18.55s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [31:17<00:00, 18.77s/it]


         Obj Val  Constraints Viol  Elapsed Time
count  27.000000         27.000000    100.000000
mean    1.936105          0.142438     18.769905
std     4.206805          0.513188     25.993760
min    -0.479532          0.000000      0.230975
25%    -0.222622          0.000000      0.767495
50%     0.288025          0.000000      2.488984
75%     2.632359          0.000000     59.744098
max    20.413122          1.923506     60.557546
Number of infeasible solution: 2
Number of None values:  73


In [7]:
df = pd.read_csv("result/nc_exact_20-20.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

         Obj Val  Constraints Viol  Elapsed Time
count  27.000000         27.000000    100.000000
mean    1.936105          0.142438     18.769905
std     4.206805          0.513188     25.993760
min    -0.479532          0.000000      0.230975
25%    -0.222622          0.000000      0.767495
50%     0.288025          0.000000      2.488984
75%     2.632359          0.000000     59.744098
max    20.413122          1.923506     60.557546
Number of infeasible solution: 2
Number of None values:  73


## Heuristic - Round

In [20]:
from src.heuristic import naive_round

In [22]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_heur_rnd_20-20.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:24:23<00:00, 50.63s/it]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000        100.000000    100.000000
mean     27.721334          0.395441     50.601916
std     279.692717          0.426800     19.300724
min      -0.574306          0.000000      2.002924
25%      -0.446714          0.184551     59.719557
50%      -0.348051          0.338304     60.233627
75%      -0.189478          0.510957     60.383489
max    2796.677544          3.471274     61.173053
Number of infeasible solution: 98


## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model_heur.solve("scip")
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/nc_heur_n1_20-20.csv")

  3%|██                                                                            | 26/1000 [01:05<1:37:32,  6.01s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 13%|█████████▊                                                                   | 127/1000 [04:04<1:07:08,  4.61s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 18%|█████████████▉                                                                 | 176/1000 [04:11<01:52,  7.35it/s]

...

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 99%|█████████████████████████████████████████████████████████████████████████████▉ | 986/1000 [17:45<00:02,  5.91it/s]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [17:47<00:00,  1.07s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  9.800000e+02        980.000000   1000.000000
mean   4.156708e+03          0.034875      1.066962
std    9.383635e+04          0.337414      7.066323
min   -3.232752e-01          0.000000      0.073751
25%    1.951013e-01          0.000000      0.093626
50%    6.638513e-01          0.000000      0.154481
75%    5.273073e+00          0.000000      0.185369
max    2.771145e+06          4.584774     60.766126
Number of infeasible solution: 13
Number of None values:  20


In [9]:
df = pd.read_csv("result/nc_heur_n1_20-20.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

           Obj Val  Constraints Viol  Elapsed Time
count    99.000000         99.000000    100.000000
mean     29.918317          0.048513      1.814759
std     244.315654          0.402934      9.494586
min      -0.177232          0.000000      0.075632
25%       0.188054          0.000000      0.116067
50%       0.947668          0.000000      0.169340
75%       4.009457          0.000000      0.191995
max    2430.918653          3.919692     60.329085
Number of infeasible solution: 2
Number of None values:  1


## Learnable Rounding

In [10]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [12]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [13]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 595.90
Epoch 1, Validation Loss: 1.45
Epoch 2, Validation Loss: 1.02
Epoch 3, Validation Loss: 1.14
Epoch 4, Validation Loss: 1.05
Epoch 5, Validation Loss: 0.86
Epoch 6, Validation Loss: 0.93
Epoch 7, Validation Loss: 0.75
Epoch 8, Validation Loss: 0.92
Epoch 9, Validation Loss: 0.82
Epoch 10, Validation Loss: 0.92
Epoch 11, Validation Loss: 0.94
Epoch 12, Validation Loss: 0.74
Epoch 13, Validation Loss: 0.74
Epoch 14, Validation Loss: 0.80
Epoch 15, Validation Loss: 0.77
Epoch 16, Validation Loss: 0.71
Epoch 17, Validation Loss: 0.70
Epoch 18, Validation Loss: 0.69
Epoch 19, Validation Loss: 0.67
Epoch 20, Validation Loss: 0.71
Epoch 21, Validation Loss: 0.57
Epoch 22, Validation Loss: 0.65
Epoch 23, Validation Loss: 0.66
Epoch 24, Validation Loss: 0.68
Epoch 25, Validation Loss: 0.67
Epoch 26, Validation Loss: 0.63
Epoch 27, Validation Loss: 0.55
Epoch 28, Validation Loss: 0.63
Epoch 29, Validation Loss: 0.67
Epoch 30, Validation Loss: 0.56
Epoch 31, Valida

In [14]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_20-20.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 124.84it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     0.397431          0.000022      0.003525
std      0.119622          0.000223      0.000854
min      0.178874          0.000000      0.001997
25%      0.321162          0.000000      0.003000
50%      0.384433          0.000000      0.003526
75%      0.461228          0.000000      0.004012
max      0.802535          0.002229      0.005953
Number of infeasible solution: 1


## Learnable Threshold

In [15]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [16]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [17]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [18]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 492.62
Epoch 1, Validation Loss: 1.12
Epoch 2, Validation Loss: 0.29
Epoch 3, Validation Loss: 0.26
Epoch 4, Validation Loss: 0.19
Epoch 5, Validation Loss: 0.31
Epoch 6, Validation Loss: 0.21
Epoch 7, Validation Loss: 0.21
Epoch 8, Validation Loss: 0.12
Epoch 9, Validation Loss: 0.17
Epoch 10, Validation Loss: 0.18
Epoch 11, Validation Loss: 0.23
Epoch 12, Validation Loss: 0.20
Epoch 13, Validation Loss: 0.11
Epoch 14, Validation Loss: 0.14
Epoch 15, Validation Loss: 0.08
Epoch 16, Validation Loss: 0.12
Epoch 17, Validation Loss: 0.17
Epoch 18, Validation Loss: 0.13
Epoch 19, Validation Loss: 0.08
Epoch 20, Validation Loss: 0.07
Epoch 21, Validation Loss: 0.08
Epoch 22, Validation Loss: 0.08
Epoch 23, Validation Loss: 0.07
Epoch 24, Validation Loss: 0.09
Epoch 25, Validation Loss: 0.30
Epoch 26, Validation Loss: 0.06
Epoch 27, Validation Loss: 0.08
Epoch 28, Validation Loss: 0.06
Epoch 29, Validation Loss: 0.09
Epoch 30, Validation Loss: 0.05
Epoch 31, Valida

In [19]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_20-20.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 153.65it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    -0.000194               0.0      0.003053
std      0.056646               0.0      0.000960
min     -0.061378               0.0      0.001996
25%     -0.038801               0.0      0.002506
50%     -0.018060               0.0      0.002998
75%      0.015285               0.0      0.003530
max      0.192843               0.0      0.006510
Number of infeasible solution: 0


## Parametric Learning Then Rounding

In [20]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [21]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [22]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [23]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 494.47
Epoch 1, Validation Loss: -0.44
Epoch 2, Validation Loss: -2.54
Epoch 3, Validation Loss: -3.12
Epoch 4, Validation Loss: -3.20
Epoch 5, Validation Loss: -3.43
Epoch 6, Validation Loss: -3.32
Epoch 7, Validation Loss: -3.43
Epoch 8, Validation Loss: -3.32
Epoch 9, Validation Loss: -3.40
Epoch 10, Validation Loss: -3.40
Epoch 11, Validation Loss: -3.24
Epoch 12, Validation Loss: -3.42
Epoch 13, Validation Loss: -3.15
Epoch 14, Validation Loss: -3.46
Epoch 15, Validation Loss: -3.65
Epoch 16, Validation Loss: -3.73
Epoch 17, Validation Loss: -3.67
Epoch 18, Validation Loss: -3.70
Epoch 19, Validation Loss: -3.74
Epoch 20, Validation Loss: -3.68
Epoch 21, Validation Loss: -3.82
Epoch 22, Validation Loss: -3.91
Epoch 23, Validation Loss: -3.85
Epoch 24, Validation Loss: -3.94
Epoch 25, Validation Loss: -3.72
Epoch 26, Validation Loss: -3.94
Epoch 27, Validation Loss: -3.87
Epoch 28, Validation Loss: -3.92
Epoch 29, Validation Loss: -3.96
Epoch 30, Validatio

In [24]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_pr_20-20.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 177.15it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     5.157895          0.042595      0.000922
std      0.433152          0.066922      0.000556
min      4.071636          0.000000      0.000000
25%      4.906361          0.000000      0.000997
50%      5.123234          0.005193      0.001000
75%      5.330060          0.059425      0.001002
max      6.796535          0.337029      0.002515
Number of infeasible solution: 52


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 492.62
Epoch 1, Validation Loss: -0.35
Epoch 2, Validation Loss: -2.16
Epoch 3, Validation Loss: -2.54
Epoch 4, Validation Loss: -2.81
Epoch 5, Validation Loss: -2.46
Epoch 6, Validation Loss: -2.43
Epoch 7, Validation Loss: -2.92
Epoch 8, Validation Loss: -2.88
Epoch 9, Validation Loss: -2.83
Epoch 10, Validation Loss: -2.92
Epoch 11, Validation Loss: -2.67
Epoch 12, Validation Loss: -2.83
Epoch 13, Validation Loss: -2.84
Epoch 14, Validation Loss: -2.91
Epoch 15, Validation Loss: -2.90
Epoch 16, Validation Loss: -2.87
Epoch 17, Validation Loss: -2.93
Epoch 18, Validation Loss: -2.83
Epoch 19, Validation Loss: -2.98
Epoch 20, Validation Loss: -2.87
Epoch 21, Validation Loss: -2.91
Epoch 22, Validation Loss: -2.93
Epoch 23, Validation Loss: -3.03
Epoch 24, Validation Loss: -2.93
Epoch 25, Validation Loss: -3.02
Epoch 26, Validation Loss: -3.04
Epoch 27, Validation Loss: -3.14
Epoch 28, Validation Loss: -3.21
Epoch 29, Validation Loss: -3.12
Epoch 30, Validatio

In [11]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_st_20-20.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 142.77it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     4.693881               0.0      0.002137
std      0.528224               0.0      0.000926
min      3.433225               0.0      0.000999
25%      4.364308               0.0      0.001503
50%      4.730306               0.0      0.002008
75%      5.053669               0.0      0.002583
max      6.162250               0.0      0.005870
Number of infeasible solution: 0
